This notebook extracts and formats several socio-demographic predictor variables by Parisian district over the period 2006-2016 (last available year). These data come from Insee's IRIS database, which collects several hundreds of variables at the sub-city level. 

We selected variables that we believe have a strong influence (potentially causal) on the outcome of elections in each district of Paris. Our assumption may be wrong, but it will be easy to see that once we put the data into the model -- it won't run or will tell us that these variables are not correlated with the outcome. The model will use these predictors to try and predict election results in each district, but we'll do that in another notebook. 

Let's start with some import statements and handy functions to extract predictors:

In [1]:
%load_ext lab_black
%load_ext watermark

import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from fbprophet import Prophet
from pathlib import Path
from typing import List

logging.getLogger().setLevel(logging.CRITICAL)

variables = {
    "activite_residents": {
        "C_ACTOCC1564": "actifs_occupes",
        "P_CHOM1564": "chomeurs",
        "P_RETR1564": "retired",
        "C_ACTOCC1564_CS3": "csp_plus",
        "C_ACTOCC1564_CS5": "csp_employed",
        "P_SAL15P_CDD": "cdd",
        "P_SAL15P_INTERIM": "interim",
        "P_SAL15P_EMPAID": "empaid",
    },
    "couples_familles_menages": {
        "P_POP2554": "pop2554",
        "P_POP5579": "pop5579",
        "C_FAMMONO": "fam_mono",
    },
    "diplomes_formation": {
        "P_NSCOL15P_SUP": "college_grad",
        "P_NSCOL15P_BAC": "non_college",
        "P_NSCOL15P_DIPLMIN": "min_diploma",
    },
    "logement": {"P_RP_100M2P": "big_house"},
    "population": {
        "P_POP1824": "youth",
        "P_POP_IMM": "immigration",
        "P_POP3044": "pop3044",
        "P_POP4559": "pop4559",
        "P_POP6074": "pop6074",
    },
}

In [2]:
def extract_predictors(repo: str) -> pd.DataFrame:
    """
    Gets all files in the given repo, selects wanted predictor variables (and takes care
    of change of perimeter in 2013 for some of them), restricts to Paris, aggregates predictors
    by district, and then returns formatted time series.
    """
    basepath = Path(f"../../../Downloads/db_iris_all/{repo}/")
    files_in_path = basepath.glob("*.xls")
    print(f"Began extracting predictors from {repo} repo...")

    # load and concat files (heavy):
    preds = []
    for file in files_in_path:
        year = file.stem[-2:]
        var_cols = extract_vars(repo, year)
        df = pd.read_excel(
            file,
            header=5,
            sheet_name="IRIS",
            usecols=["DEP", "LIBCOM"] + var_cols,
            dtype={"DEP": "category", "LIBCOM": "category"},
            nrows=40_500,
        )
        df = df[df.DEP == "75"].reset_index(drop=True).drop("DEP", axis=1)
        # handle change of perimeter in data:
        if ((repo == "diplomes_formation") and (year <= "12")) or (
            (repo == "logement") and (year > "12")
        ):
            df = reconcile_perimeter(repo, year, var_cols, df)
        preds.append(df)
    preds = pd.concat(preds, axis=1)

    return agg_and_format(repo, preds)


def extract_vars(repo: str, year: str) -> List:
    """
    From the repo and year, make a list of appropriate variables to extract from the file.
    The perimeter of sampled data changed in 2013 for two categories of variables we're interested in
    (diplomes_formation and logement) -- the function handles that by selecting the right columns for 
    each year.
    """
    var_map = variables[repo]
    var_cols = [f"{var_code[:1]}{year}{var_code[1:]}" for var_code in var_map.keys()]
    # handle change of perimeter in data:
    if (repo == "diplomes_formation") and (year <= "12"):
        var_cols = [
            f"P{year}_NSCOL15P_{v}"
            for v in ["DIPL0", "CEP", "BEPC", "BAC", "BACP2", "SUP"]
        ]
    if (repo == "logement") and (year > "12"):
        var_cols = [f"P{year}_RP_{v}" for v in ["100120M2", "120M2P"]]

    return var_cols


def reconcile_perimeter(
    repo: str, year: str, var_cols: List, df: pd.DataFrame
) -> pd.DataFrame:
    """
    This function reconciles the change of perimeter that occured in 2013 for variables in 
    diplomes_formation and logement. It just sums the appropriate columns to get the same 
    perimeter for all years, and then drops the useless columns from the dataframe.
    """
    if (repo == "diplomes_formation") and (year <= "12"):
        df[f"P{year}_NSCOL15P_DIPLMIN"] = df[
            [
                f"P{year}_NSCOL15P_DIPL0",
                f"P{year}_NSCOL15P_CEP",
                f"P{year}_NSCOL15P_BEPC",
            ]
        ].sum(1)
        df[f"P{year}_NSCOL15P_SUP"] = df[
            [f"P{year}_NSCOL15P_BACP2", f"P{year}_NSCOL15P_SUP"]
        ].sum(1)
        df = df.drop(
            [
                f"P{year}_NSCOL15P_DIPL0",
                f"P{year}_NSCOL15P_CEP",
                f"P{year}_NSCOL15P_BEPC",
                f"P{year}_NSCOL15P_BACP2",
            ],
            axis=1,
        )
    if (repo == "logement") and (year > "12"):
        df[f"P{year}_RP_100M2P"] = df[var_cols].sum(1)
        df = df.drop(var_cols, axis=1)

    return df


def agg_and_format(repo: str, df: pd.DataFrame) -> pd.DataFrame:
    """
    Takes the raw timeseries of predictors, aggregates them by district,
    prettifies columns and returns timeseries of all predictors in
    appropriate format.
    """
    # drop duplicated column values:
    df = df.T.drop_duplicates().T
    # drop duplicated column names:
    df = df.loc[:, ~df.columns.duplicated()]

    # extract district number:
    df["LIBCOM"] = df.LIBCOM.str.extract("(\d+)").astype(int)
    df = df.rename(columns={"LIBCOM": "arrondissement"})

    # aggregate by district and prettify columns:
    df = df.groupby("arrondissement").sum()
    temp = []
    for var_code, var_name in variables[repo].items():
        print(f"Formatting {var_name} ({var_code})...")
        dat = df.loc[:, df.columns.str.endswith(var_code[2:])]
        dat.columns = dat.columns.str[1:3].astype(int) + 2000
        dat.columns.name = "year"
        dat = dat.sort_index(axis=1).stack()
        dat.name = var_name
        temp.append(dat)

    print(f"Finished extracting and aggregating predictors.\n")
    return pd.concat(temp, axis=1)

The raw excel files where the data live are very heavy, so this function will take some time to run -- but it will be worth it. Indeed, it will go and load the files where each predictor is, for  each year on record, do some formatting and restricting and then return a dataframe with the proper time series. Let's run it and go get a cup of coffee:

In [3]:
%%time
predictors = []
for repo in variables.keys():
    predictors.append(extract_predictors(repo))

Began extracting predictors from activite_residents repo...
Formatting actifs_occupes (C_ACTOCC1564)...
Formatting chomeurs (P_CHOM1564)...
Formatting retired (P_RETR1564)...
Formatting csp_plus (C_ACTOCC1564_CS3)...
Formatting csp_employed (C_ACTOCC1564_CS5)...
Formatting cdd (P_SAL15P_CDD)...
Formatting interim (P_SAL15P_INTERIM)...
Formatting empaid (P_SAL15P_EMPAID)...
Finished extracting and aggregating predictors.

Began extracting predictors from couples_familles_menages repo...
Formatting pop2554 (P_POP2554)...
Formatting pop5579 (P_POP5579)...
Formatting fam_mono (C_FAMMONO)...
Finished extracting and aggregating predictors.

Began extracting predictors from diplomes_formation repo...
Formatting college_grad (P_NSCOL15P_SUP)...
Formatting non_college (P_NSCOL15P_BAC)...
Formatting min_diploma (P_NSCOL15P_DIPLMIN)...
Finished extracting and aggregating predictors.

Began extracting predictors from logement repo...
Formatting big_house (P_RP_100M2P)...
Finished extracting and ag

In [4]:
predictors = pd.concat(predictors, axis=1)
predictors

actifs_occupes      chomeurs      retired      csp_plus  \
arrondissement year                                                            
1              2006     9485.059228    895.724765   430.144083   4739.436591   
               2007     9546.148694    922.094508   484.611909   4886.163367   
               2008     9469.633224    883.164655   434.413766   5042.215106   
               2009     9665.691628    915.091495   423.110985   5121.184955   
               2010     9558.180760    888.276570   388.643955   5126.709956   
...                             ...           ...          ...           ...   
20             2012    91753.677270  15853.711588  6533.212180  32014.599237   
               2013    90488.610079  16120.353453  5951.525911  31897.426529   
               2014    90469.181326  16205.968043  5691.641883  32923.619569   
               2015    90370.240523  16233.260643  5153.572477  33664.134135   
               2016    90874.227479  16037.251736  4780.503292  34691.926164   

                     csp_employed           cdd      interim      empaid  \
arrondissement year                                                        
1              2006   1701.623443   1012.695824    32.424709   29.045563   
               2007   1647.520281   1074.968469    24.481698   41.166505   
               2008   1486.566750    993.190195    25.239743   32.648352   
               2009   1578.802223    985.089081    31.114696   15.404247   
               2010   1554.013248    959.031002    32.949848   13.047890   
...                           ...           ...          ...         ...   
20             2012  22257.487451  11003.343903  1038.460995  459.866692   
               2013  21875.419589  10777.232752   930.068787  504.964839   
               2014  21385.380246  10532.548831   965.810470  510.035432   
               2015  20587.795831  10370.859531   994.985599  548.623173   
               2016  20514.362237  10335.255850   964.492112  529.753550   

                          pop2554       pop5579      fam_mono  college_grad  \
arrondissement year                                                           
1              2006   8849.957510   3858.176771    482.374749   7890.884720   
               2007   8915.091416   3983.321277    494.126055   8241.363122   
               2008   8845.308159   3726.960725    473.505964   8241.961872   
               2009   8967.554268   3747.496244    520.455533   8471.428572   
               2010   8886.736184   3600.179553    509.405538   8415.575368   
...                           ...           ...           ...           ...   
20             2012  91218.784981  41270.667858  10929.308718  63510.792515   
               2013  90141.737553  41134.744292  10908.102652  63058.398038   
               2014  90139.233180  42214.033848  10969.012051  66282.764695   
               2015  89863.300140  43037.092283  10753.108869  68786.240273   
               2016  90067.474203  43475.035516  10632.698061  71851.173023   

                      non_college   min_diploma    big_house         youth  \
arrondissement year                                                          
1              2006   1897.393615   2623.277344  1252.732849   1874.672223   
               2007   1860.338159   2475.048769  1286.164264   1866.646378   
               2008   1834.442888   2245.159195  1216.472899   1816.180756   
               2009   1852.049112   2248.727240  1302.325425   1842.097989   
               2010   1922.670888   2062.217398  1397.237101   1779.033595   
...                           ...           ...          ...           ...   
20             2012  22814.414012  39510.527039  3789.992165  18234.641207   
               2013  22184.345103  38797.154476  3792.380980  18156.671990   
               2014  21901.915225  37070.754157  3850.537722  18133.119561   
               2015  21547.794958  35383.166953  3843.909770  17977.773858   
               2016  20939.

Had a nice coffee? As you can see, we now have the predictors ready to match with past election results, and then to give to the model! Ready? Well, not completely... The data stop in 2016 but we will train our model on elections as recent as 2017, and we'll test it on 2019 European elections, so we need data for the period 2017-2019.

Unfortunately, this type of data generally takes two years to produce. This means 2019 data should be available around 2021 -- we can't wait for that long! Facebook's Prophet library comes very handy here and will allow us to make some reasonable extrapolations of the predictors' values. Ideally, we should think hard about Prophet's default settings and if they are adapted to our use case -- we could even see if our predictors could be predicted by other, available data.

Here however, I'll do a quick and dirty extrapolation, sticking to Prophet's default. We'll see how the model handles that and we can always do better afterwards if needed. Actually, I think it could be even more helpful to incorporate measurement error on predictors *into* the model, so that the Bayesian machinery takes it into account -- so let's not spend too much time here, at least for our first iteration.

Let's turn our `year` variable into a real datetime (new year's eve) and write our interpolation function:

In [5]:
predictors = predictors.reset_index().set_index("arrondissement")
predictors["year"] = pd.to_datetime(predictors.year, format="%Y") + pd.DateOffset(
    months=11, days=30
)
predictors

,year,actifs_occupes,chomeurs,retired,csp_plus,csp_employed,cdd,interim,empaid,pop2554,...,fam_mono,college_grad,non_college,min_diploma,big_house,youth,immigration,pop3044,pop4559,pop6074
arrondissement,,,,,,,,,,,,,,,,,,,,,
1,2006-12-31,9485.059228,895.724765,430.144083,4739.436591,1701.623443,1012.695824,32.424709,29.045563,8849.957510,...,482.374749,7890.884720,1897.393615,2623.277344,1252.732849,1874.672223,3148.134542,4707.453445,3416.928896,2104.779557
1,2007-12-31,9546.148694,922.094508,484.611909,4886.163367,1647.520281,1074.968469,24.481698,41.166505,8915.091416,...,494.126055,8241.363122,1860.338159,2475.048769,1286.164264,1866.646378,3227.921219,4817.630783,3442.936791,2267.841713
1,2008-12-31,9469.633224,883.164655,434.413766,5042.215106,1486.566750,993.190195,25.239743,32.648352,8845.308159,...,473.505964,8241.961872,1834.442888,2245.159195,1216.472899,1816.180756,3121.358408,4818.893498,3220.943691,2196.234294
1,2009-12-31,9665.691628,915.091495,423.110985,5121.184955,1578.802223,985.089081,31.114696,15.404247,8967.554268,...,520.455533,8471.428572,1852.049112,2248.727240,1302.325425,1842.097989,3121.406343,4852.458184,3198.773394,2221.642431
1,2010-12-31,9558.180760,888.276570,388.643955,5126.709956,1554.013248,959.031002,32.949848,13.047890,8886.736184,...,509.405538,8415.575368,1922.670888,2062.217398,1397.237101,1779.033595,3021.113668,4792.879306,3134.694812,2092.698608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,2012-12-31,91753.677270,15853.711588,6533.212180,32014.599237,22257.487451,11003.343903,1038.460995,459.866692,91218.784981,...,10929.308718,63510.792515,22814.414012,39510.527039,3789.992165,18234.641207,43045.904247,48150.370414,38491.637337,24506.293474
20,2013-12-31,90488.610079,16120.353453,5951.525911,31897.426529,21875.419589,10777.232752,930.068787,504.964839,90141.737553,...,10908.102652,63058.398038,22184.345103,38797.154476,3792.380980,18156.671990,42888.160363,47306.558725,38119.996809,24641.667894
20,2014-12-31,90469.181326,16205.968043,5691.641883,32923.619569,21385.380246,10532.548831,965.810470,510.035432,90139.233180,...,10969.012051,66282.764695,21901.915225,37070.754157,3850.537722,18133.119561,42123.803538,46727.667351,38384.638482,25714.740471


In [6]:
def extrapol_pred(
    district: int, predictor: str, pred_df: pd.DataFrame, timeframe: int
) -> pd.DataFrame:
    """
    Quick and dirty extrapolation of predictor in the district, for the number of years 
    specified in timeframe variable. The function uses Facebook's Prophet default 
    settings -- hence 'quick and dirty'.
    """
    df = pred_df.loc[district, ["year", predictor]].reset_index(drop=True)
    df.columns = ["ds", "y"]  # Prophet needs this names

    m = Prophet()
    m.fit(df)
    future = m.make_future_dataframe(periods=timeframe, freq="Y")
    forecast = m.predict(future)

    forecast = forecast.iloc[-timeframe:][["ds", "yhat"]]
    forecast.columns = ["year", predictor]

    forecast.index = [district] * len(forecast)
    forecast.index.name = "arrondissement"
    forecast = forecast.reset_index().set_index(["arrondissement", "year"])
    return forecast

Each pair (district, predictor) represents a time series that we extrapolate over the next three years (2017-2019). Then, we combine all that in a dataframe:

In [7]:
%%time
districts_dfs = []

for district in predictors.index.unique():
    extrapol = []
    for predictor in predictors.columns.difference(["year"]):
        extrapol.append(extrapol_pred(district, predictor, predictors, timeframe=3))
    
    print(f"Finished extrapolating all predictors for district {district}\n")
    districts_dfs.append(pd.concat(extrapol, axis=1))

districts_dfs = pd.concat(districts_dfs)

Finished extrapolating all predictors for district 1

Finished extrapolating all predictors for district 2

Finished extrapolating all predictors for district 3

Finished extrapolating all predictors for district 4

Finished extrapolating all predictors for district 5

Finished extrapolating all predictors for district 6

Finished extrapolating all predictors for district 7

Finished extrapolating all predictors for district 8

Finished extrapolating all predictors for district 9

Finished extrapolating all predictors for district 10

Finished extrapolating all predictors for district 11

Finished extrapolating all predictors for district 12

Finished extrapolating all predictors for district 13

Finished extrapolating all predictors for district 14

Finished extrapolating all predictors for district 15

Finished extrapolating all predictors for district 16

Finished extrapolating all predictors for district 17

Finished extrapolating all predictors for district 18

Finished extrapolat

The only thing left to do is concatenating the extrapolations and the observed data:

In [8]:
predictors = pd.concat(
    [predictors.reset_index().set_index(["arrondissement", "year"]), districts_dfs],
    sort=True,
).sort_index()
predictors.to_csv("data/predictors_by_district.csv")
predictors

actifs_occupes    big_house           cdd  \
arrondissement year                                                    
1              2006-12-31     9485.059228  1252.732849   1012.695824   
               2007-12-31     9546.148694  1286.164264   1074.968469   
               2008-12-31     9469.633224  1216.472899    993.190195   
               2009-12-31     9665.691628  1302.325425    985.089081   
               2010-12-31     9558.180760  1397.237101    959.031002   
...                                   ...          ...           ...   
20             2015-12-31    90370.240523  3843.909770  10370.859531   
               2016-12-31    90874.227479  3948.492908  10335.255850   
               2017-12-31    90936.701645  3968.190376   9943.758332   
               2018-12-31    90917.272892  4025.603898   9732.150904   
               2019-12-31    90818.332088  4085.150428   9579.350096   

                               chomeurs  college_grad  csp_employed  \
arrondissement year                                                   
1              2006-12-31    895.724765   7890.884720   1701.623443   
               2007-12-31    922.094508   8241.363122   1647.520281   
               2008-12-31    883.164655   8241.961872   1486.566750   
               2009-12-31    915.091495   8471.428572   1578.802223   
               2010-12-31    888.276570   8415.575368   1554.013248   
...                                 ...           ...           ...   
20             2015-12-31  16233.260643  68786.240273  20587.795831   
               2016-12-31  16037.251736  71851.173023  20514.362237   
               2017-12-31  16571.496502  75777.953185  19995.235337   
               2018-12-31  16859.038980  79002.319842  19482.271525   
               2019-12-31  17081.428057  81505.795420  18895.056111   

                               csp_plus      empaid      fam_mono  \
arrondissement year                                                 
1              2006-12-31   4739.436591   29.045563    482.374749   
               2007-12-31   4886.163367   41.166505    494.126055   
               2008-12-31   5042.215106   32.648352    473.505964   
               2009-12-31   5121.184955   15.404247    520.455533   
               2010-12-31   5126.709956   13.047890    509.405538   
...                                 ...         ...           ...   
20             2015-12-31  33664.134135  548.623173  10753.108869   
               2016-12-31  34691.926164  529.753550  10632.698061   
               2017-12-31  34330.223246  438.795993  10671.081485   
               2018-12-31  34691.789173  435.576125  10623.017339   
               2019-12-31  35366.612459  431.995141  10497.678014   

                            immigration     interim   min_diploma  \
arrondissement year                                                 
1              2006-12-31   3148.134542   32.424709   2623.277344   
               2007-12-31   3227.921219   24.481698   2475.048769   
               2008-12-31   3121.358408   25.239743   2245.159195   
               2009-12-31   3121.406343   31.114696   2248.727240   
               2010-12-31   3021.113668   32.949848   2062.217398   
...                                 ...         ...           ...   
20             2015-12-31  41633.325845  994.985599  35383.166953   
               2016-12-31  41180.244725  964.492112  33556.709791   
               2017-12-31  40830.947452  870.453890  31784.431697   
               2018-12-31  40333.333716  819.132293  30058.031378   
               2019-12-31  39652.934880  770.275084  28370.444174   

                            non_college       pop2554       pop3044  \
arrondissement year                                                   
1              2006-12-31   1897.393615   8849.957510   4707.453445   
               2007-12-31   1860.338159   8915.091416   4817.630783   
               2008-12-31   1834.442888   8845.308159   4818.893498   
               

And now we're ready to match predictors against past election results, and to give data to the model! Let's do that in another notebook.

In [9]:
%watermark -a AlexAndorra -n -u -v -iv

numpy   1.17.3
pandas  0.25.3
seaborn 0.9.0
logging 0.5.1.2
AlexAndorra 
last updated: Wed Nov 27 2019 

CPython 3.7.5
IPython 7.9.0
